In [ ]:
import pandas as pd 
import pickle
import datetime
import re
import networkx as nx
import numpy as np
from tqdm import tqdm

# Atrial fibrillation (AF) patients on rivaroxaban

In [ ]:
with open('/projects/ROCKET AF/1A/rivaroxaban dataextraction/data/first_mention_rivaroxaban.pickle', 'rb') as f:
    rivaroxaban = pickle.load(f)

In [ ]:
len(rivaroxaban)

In [ ]:
df = pd.DataFrame(list(rivaroxaban.items()), columns=['client_idcode', 'first_mention_rivaroxaban']) # This code converts a dictionary (rivaroxaban) into a Pandas DataFrame with two columns named 'client_idcode' and 'first_mention_rivaroxaban'
df.shape

In [ ]:
df.head()

In [ ]:
ids = set(df['client_idcode'])
len(ids)

# AF patients on rivaroxaban with atrial myxome or left ventricular thrombus

In [ ]:
def graph_from_onto(onto):
    G = nx.DiGraph()
    cl_edges = []
    for s, ts in onto.items():
        for t in ts:
            cl_edges.append((s.replace('S-', ''), t.replace('S-', '')))
    G.add_edges_from(cl_edges)
    return G

def load_onto(o):
    with open(f'/projects/data/GS/{o}', 'rb') as f:
        onto = pickle.load(f)
    G = graph_from_onto(onto)
    return G

def expand_codes(onto, codes):
    expanded = {}
    for name, top_codes in codes.items():
        expanded[name] = set()
        for c in top_codes:
            expanded[name].add(c)
            if c in onto:
                expanded[name].update(nx.ancestors(onto, c))
            else:
                print("NOT FOUND:", c, name)
        print(name, len(top_codes), len(expanded[name]))
    return expanded

In [ ]:
onto = load_onto('isa_rela_ch2pt_202009.pickle')

In [ ]:
db = {'atrial_myxoma' : ['233854003'],
      'left_ventricular_thrombus' : ['309519009']
     }
db = expand_codes(onto,db)

In [ ]:
%%time 
with open('/projects/data/GS/pt2cui_pos_dates.pickle', 'rb') as f:
    pt2cui_pos_dates = pickle.load(f)

In [ ]:
def dates_ex(pt_data, codes):
    dates = None
    for x in codes:
        if x in pt_data and len(pt_data[x])>=2:
            if dates == None:
                dates = pt_data[x]
            else: 
                dates = dates.union(pt_data[x])
    return dates

rows = []
for pt in ids:
    pt_data = pt2cui_pos_dates.get(pt, {})
    row = {'client_idcode': pt, 'first_mention_rivaroxaban': rivaroxaban[pt]} 
    for concept, codes in db.items():
        dates = dates_ex(pt_data, codes)
        if dates == None:
            row[f"{concept}_date"] = np.nan
        else:
            row[f"{concept}_date"] = dates
    rows.append(row)

In [ ]:
df = pd.DataFrame(rows)
df.shape

In [ ]:
df.head()

In [ ]:
df.dropna(subset=['atrial_myxoma_date', 'left_ventricular_thrombus_date'], how='all', inplace = True)

In [ ]:
df.head()

In [ ]:
df['client_idcode'].nunique()

In [ ]:
%%time
df = df[['client_idcode']].join((df[i].explode() for i in df.iloc[:,1:]))
df.shape

In [ ]:
df.head()

In [ ]:
for k in db.keys():
    df[f'{k}_date'] = pd.to_datetime(df[f'{k}_date']).dt.date

In [ ]:
df.head()

In [ ]:
for k in db.keys():
    df[f'{k}_delta'] = pd.to_datetime(df['first_mention_rivaroxaban']) - pd.to_datetime(df[f'{k}_date'])

In [ ]:
df.head()

In [ ]:
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
for k in db.keys():
    t1 = df[f'{k}_delta'] >= min_time 
    t2 = df[f'{k}_delta'] <= max_time
    in_window = t1 & t2 
    df[f'{k}_in_window'] = in_window

In [ ]:
df.head()

In [ ]:
atrial_myxoma = df.loc[df['atrial_myxoma_in_window'] == True]
atrial_myxoma = atrial_myxoma[['client_idcode', 'atrial_myxoma_in_window']]
atrial_myxoma.shape

In [ ]:
atrial_myxoma = set(atrial_myxoma['client_idcode'])
len(atrial_myxoma)

In [ ]:
with open('/projects/ROCKET AF/1A/rivaroxaban dataextraction/data/atrial_myxoma.pickle', 'wb') as f:
    pickle.dump(atrial_myxoma,f)

In [ ]:
left_ventricular_thrombus = df.loc[df['left_ventricular_thrombus_in_window'] == True]
left_ventricular_thrombus = left_ventricular_thrombus[['client_idcode', 'left_ventricular_thrombus_in_window']]
left_ventricular_thrombus.shape

In [ ]:
df.loc[df['left_ventricular_thrombus_delta'].notna()]

In [ ]:
df.loc[df['left_ventricular_thrombus_delta'].notna()].shape